In [ ]:
# Setting a random seed for reproducible results
import random
random.seed(42)

import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
from PIL import Image
import random
import openslide as op
import openslide.deepzoom as deepzoom
import cv2


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D,GlobalAveragePooling2D, BatchNormalization                                    
from tensorflow.keras import utils, regularizers
from tensorflow.keras.callbacks import EarlyStopping, History
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras import utils

In [ ]:
TARGETS = ['normal', 'squamous', 'adenocarcinoma']

train_dir = '/users/seanfrey/desktop/Train/'
test_dir = '/users/seanfrey/desktop/Test/'


In [ ]:
training_data = []
def create_training_data():
    for target in TARGETS:
        path = os.path.join(train_dir, target)
        class_num = TARGETS.index(target)
        for img in os.listdir(path):
            try:
                # Open the file and deep zoom generator.
                slide = op.OpenSlide(os.path.join(path, img))
                tile = op.deepzoom.DeepZoomGenerator(osr = slide, tile_size = 8640, overlap = 0)
                x, y = tile.level_tiles[11]
                im = tile.get_tile(11, (x-1, y-1))
                
                a,b,c = np.array(im).shape
                x2 = (b//2) - 300
                y2 = (a//2) - 300
                x3 = (b//2) + 300
                y3 = (a//2) + 300
                
                img_array = np.array(im.crop((x2, y2, x3, y3))).astype('float32')
                training_data.append([img_array, class_num])
            
            except:
                continue
                
create_training_data()

In [ ]:
testing_data = []
def create_testing_data():
    for target in TARGETS:
        path = os.path.join(test_dir, target)
        class_num = TARGETS.index(target)
        for img in os.listdir(path):
            try:
                # Open the file and deep zoom generator.
                slide = op.OpenSlide(os.path.join(path, img))
                tile = op.deepzoom.DeepZoomGenerator(osr = slide, tile_size = 8640, overlap = 0)
                x, y = tile.level_tiles[11]
                im = tile.get_tile(11, (x-1, y-1))
                
                a,b,c = np.array(im).shape
                x2 = (b//2) - 300
                y2 = (a//2) - 300
                x3 = (b//2) + 300
                y3 = (a//2) + 300
                
                img_array = np.array(im.crop((x2, y2, x3, y3))).astype('float32')
                testing_data.append([img_array, class_num])
                
            except:
                continue
            
create_testing_data()

In [ ]:
X_train = []
y_train = []

for features, label in training_data:
    X_train.append(features)
    y_train.append(label)
    
X_train = np.array(X_train).reshape(-1, 600, 600, 3)

In [ ]:
X_test = []
y_test = []

for features, label in testing_data:
    X_test.append(features)
    y_test.append(label)
    
X_test = np.array(X_test).reshape(-1, 600, 600, 3)

In [ ]:
model_compressed = Sequential()

model_compressed.add(Conv2D(
    filters = 16,
    kernel_size = (3,3),
    activation = "relu",
    input_shape = (600, 600, 3)
))

model_compressed.add(MaxPooling2D(pool_size = (2,2)))

model_compressed.add(Conv2D(
    filters = 64, 
    kernel_size = (2,2),
    activation = "relu"))

model_compressed.add(MaxPooling2D(
    pool_size = (2,2)
))

model_compressed.add(Flatten())

model_compressed.add(Dense(
    128, 
    activation = "relu",
    kernel_regularizer = regularizers.l2(0.5)
))

model_compressed.add(Dropout(0.5))

model_compressed.add(Dense(
    3, 
    activation = "softmax",
    kernel_regularizer = regularizers.l2(0.5)
))

es = EarlyStopping(
    monitor = "val_loss",
    min_delta = 0,
    patience = 5
)

model_compressed.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['sparse_categorical_accuracy']
)


In [1]:
history_compressed = model_compressed.fit(
        X_train,
        y_train,
        batch_size = 4,
        validation_data = (X_test, y_test),
        epochs = 10,
        callbacks = [es],
        verbose = 1,
)

/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/Caskroom/miniconda/base/lib/python3